In [4]:
import os
import pandas as pd
import numpy as np
import csv
from collections import defaultdict
import urllib
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import gffpandas.gffpandas as gffpd  
from pyfaidx import Fasta
from Bio import SeqIO
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

2024-04-05 15:01:19.920966: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 15:01:19.987229: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 15:01:19.987261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 15:01:19.988301: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-05 15:01:19.996101: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 15:01:19.996661: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [6]:
species_names_minus_S = [
    'Acidomyces_richmondensis', 'Acidomyces_sp', 'Alternaria_alternata', 'Ascoidea_rubescens',
    'Aspergillus_brasiliensis', 'Aspergillus_candidus', 'Aspergillus_carbonarius', 'Aspergillus_glaucus',
    'Aspergillus_luchuensis', 'Aspergillus_novofumigatus', 'Aspergillus_ochraceoroseus', 'Aspergillus_sydowi',
    'Aspergillus_tubingensis', 'Aspergillus_versicolor', 'Aspergillus_wentii', 'Babjeviella_inositovora',
    'Coniochaeta_ligniaria', 'Cyberlindnera_jadinii', 'Daldinia_sp', 'Hyphopichia_burtonii', 'Hypoxylon_sp',
    'Metschnikowia_bicuspidata', 'Microdochium_bolley', 'Nadsonia_fulvescens', 'Paraphaeosphaeria_sporulosa',
    'Penicilliopsis_zonata', 'Phialocephala_scopiformis', 'Pseudomassariella_vexata', 'Pyrenochaeta_sp', 
    'Stagonospora_sp','Suhomyces_tanzawaensis', 'Xylona_heveae']




species_names = species_names_minus_S.copy()
species_names.insert(29, 'Saccharomyces_cerevisiae')


In [7]:
def read_gff3_df_list(folder, fnames):
    '''Read gff3 files to list of data frames'''
    gff3_list = []
    for fname in fnames:
        # Correctly concatenate folder path and filename
        annotation = gffpd.read_gff3(folder + fname + ".gff3")
        gff_df = annotation.df
        gff3_list.append(gff_df)
    return gff3_list

folder = 'gff/'
gff3_list = read_gff3_df_list(folder, species_names_minus_S)



In [8]:
# Handling Saccharomyces Cerevisiae data Manually

def read_gff3_df(species_name):
    '''Read a single gff3 file from the 'gff3' folder into a DataFrame'''
    # Construct the full file path including the folder name and file extension
    file_path = f"gff/{species_name}.gff3"
    annotation = gffpd.read_gff3(file_path)
    gff_df = annotation.df
    
    return gff_df

species_name = 'Saccharomyces_cerevisiae'
sac_gff3_df = read_gff3_df(species_name) # Read the GFF3 file for the given species from the 'gff3' folder

In [5]:
def extract_longest_names(gff3):
    '''extract mrna_id to gene_id data + groupby select longest transcript'''
    id_convert = pd.DataFrame(columns=['gene_id','mrna_id','mrna_len','mrna_idx']) #makes new df with the 4 columns
    select = gff3[gff3.type == 'mRNA']  #uses the inputted df and makes a new df (named 'select') where only rows with the type == mRNA are kept, along with all the other columns and data
    id_convert['mrna_id'] = select.attributes.str.split(';').apply(lambda x: x[0].split(':')[1]) # fills the mrna_id column with data from the select df 
    id_convert['gene_id'] = select.attributes.str.split(';').apply(lambda x: x[1].split(':')[1]) # same but for the gene_id 
    id_convert['mrna_len'] = np.absolute(select.start - select.end) #using the start and end columns in 'select', mrna_len is calculated and deposited into 'id_convert' df
    id_convert['mrna_idx'] = id_convert.index #basically labels the row as to its position in the df, for example first row will have an index of 0, 2nd - 1 and so on
    id_convert = id_convert.reset_index(drop=True) #ensures old index is not added (more of an error preventing line)
    id_convert_idxmax = id_convert.groupby(['gene_id'])['mrna_len'].idxmax() #one gene_id can have multiple mrna rows, this groups them
    id_convert_max = id_convert.iloc[id_convert_idxmax.values] #keeps longest mrna for each gene_id discards the rest 
    
    return id_convert_max


# Output the longest_transcripts_df DataFrame to a text file
# gff3_max = extract_longest_names(gff3_list[0]) # Printing the resulting DataFrame to view it
# print(gff3_max)

In [6]:
def group_regions_list(gff3_max,c_name):
    '''obtain regions as list from dframe'''
    #dfcopy = gff3_max.copy(deep=True)
    dfcopy = gff3_max.assign(limits_list = gff3_max[gff3_max.type == c_name][['start','end']].apply(lambda x: x.values,axis=1)) #(lambda x: x.tolist(),axis=1)
    # print(dfcopy)
    out = dfcopy[dfcopy.type == c_name].groupby('id')['limits_list'].agg(list)
    return out

#outp = group_regions_list(gff3_max, 'mRNA')
#print(outp)


In [7]:
def parse_gff3_to_regions(gff3_list):
    '''return regions dframe from gff3 data'''
    regions_list = []
        
    for gff3 in gff3_list:
        
        gff3 = gff3[gff3.type.isin(['gene','mRNA','three_prime_UTR','exon','CDS','five_prime_UTR'])] #only keeps the described columns from the inputted df from gff3_list
        
        # initialize
        id_convert_max = extract_longest_names(gff3) #takes df from gff3_list and performs function on it, labels it id_convert_max
        dframe = pd.DataFrame(columns=['gene_id','mrna_id','cds_list','5utr_list','3utr_list','ostart','oend','strand','seq_id']) #makes a new dframe with the following columns, called dframe
        
        # make id from attributes
        gff3['id'] = gff3.attributes.str.split(';').apply(lambda x: x[0].split(':')[1]) #extracts the value after the first colon encountered in the attributes string, adds it to new 'id' column
        
        # replace CDS ids with transcript ids
        gff3.loc[gff3.type == 'CDS','id'] = gff3[gff3.type == 'CDS'].attributes.str.split(';').apply(lambda x: x[1].split(':')[1])  #extracts value after second : (transcript ID)
        gff3_max = gff3[gff3.id.isin(id_convert_max.mrna_id)]
        dframe['ostart'] = gff3_max[gff3_max.type == 'mRNA'].start
        dframe['oend'] = gff3_max[gff3_max.type == 'mRNA'].end
        dframe['strand'] = gff3_max[gff3_max.type == 'mRNA'].strand
        dframe['seq_id'] = gff3_max[gff3_max.type == 'mRNA'].seq_id
        # extract ids, convert gene_id from mrna_id
        dframe['mrna_id'] = gff3_max[gff3_max.type == 'mRNA'].id
        dframe['gene_id'] = id_convert_max.set_index(['mrna_id']).loc[dframe.mrna_id.values].gene_id.values
        # use function to list to obtain lists of CDS and UTRs
        # Obtain the DataFrame with UTR lists and ensure 'mrna_id' is a column
        
        # Assuming group_regions_list(gff3_max,'five_prime_UTR') returns a Series with mrna_id as the index
        
        dframe['cds_list'] = group_regions_list(gff3_max,'CDS').loc[dframe.mrna_id].values
        five_prime_utr_series = group_regions_list(gff3_max, 'five_prime_UTR')# Map the mrna_id in dframe to this Series, filling with NaN where there's no match
        dframe['5utr_list'] = dframe['mrna_id'].map(five_prime_utr_series)
        three_prime_utr_series = group_regions_list(gff3_max, 'three_prime_UTR')
        dframe['3utr_list'] = dframe['mrna_id'].map(three_prime_utr_series)
        #print(gff3_max)


         # clear up rows with empty cds - these are not genes
        dframe.dropna(subset=['cds_list','ostart','oend','5utr_list','3utr_list'],inplace=True)

        # promoter and terminator lists
        # define boundary lists, based on strand
        # divide frame into positive and negative strand = vectorized on both halves
        len_prom = int(1000)
        len_term = int(500)
        len_prom_f = [int(1000),int(500)]
        len_term_f = [int(500),int(500)]
        # positive strand
        ostart = dframe[dframe.strand=='+'].ostart
        oend = dframe[dframe.strand=='+'].oend
        dframe.loc[dframe.strand=='+','prom_list'] = pd.concat([ostart-len_prom+1,ostart],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
        dframe.loc[dframe.strand=='+','term_list'] = pd.concat([oend,oend+len_term-1],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
        dframe.loc[dframe.strand=='+','prom_full_list'] = pd.concat([ostart-len_prom_f[0]+1,ostart+len_prom_f[1]],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
        dframe.loc[dframe.strand=='+','term_full_list'] = pd.concat([oend-len_term_f[0],oend+len_term_f[1]-1],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
        # negative strand reversed names and sizes
        ostart = dframe[dframe.strand=='-'].ostart
        oend = dframe[dframe.strand=='-'].oend
        dframe.loc[dframe.strand=='-','term_list'] = pd.concat([ostart-len_term+1,ostart],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
        dframe.loc[dframe.strand=='-','prom_list'] = pd.concat([oend,oend+len_prom-1],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
        dframe.loc[dframe.strand=='-','term_full_list'] = pd.concat([ostart-len_term_f[1]+1,ostart+len_term_f[0]],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
        dframe.loc[dframe.strand=='-','prom_full_list'] = pd.concat([oend-len_prom_f[1],oend+len_prom_f[0]-1],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])

        # calculate lengths of regions
        dframe['cds_len'] = dframe['cds_list'].apply(lambda x: sum(np.diff(x))[0])
        dframe['5utr_len'] = dframe['5utr_list'].apply(lambda x: 0 if np.any(pd.isna(x)) else sum(np.diff(x))[0])
        dframe['3utr_len'] = dframe['3utr_list'].apply(lambda x: 0 if np.any(pd.isna(x)) else sum(np.diff(x))[0])
        
        # remove regions above limit for speed
        dframe = dframe[(dframe.cds_len < 20000) & (dframe['5utr_len'] < 10000) & (dframe['3utr_len'] < 10000)].reset_index(drop=True)
        
        # assertions 
        #print(len(dframe['gene_id'].unique()) == len(dframe['gene_id']))
        #print(dframe.isnull().values.any())
        
        regions_list.append(dframe)
    return regions_list

species_dataframes = parse_gff3_to_regions(gff3_list)
print(len(species_dataframes))
print(species_dataframes)



32
[               gene_id   mrna_id  \
0     M433DRAFT_148561  KYG50636   
1     M433DRAFT_178252  KYG50640   
2     M433DRAFT_178181  KYG50642   
3     M433DRAFT_148570  KYG50643   
4     M433DRAFT_148572  KYG50644   
...                ...       ...   
4471  M433DRAFT_159963  KYG40758   
4472  M433DRAFT_532884  KYG40750   
4473  M433DRAFT_532900  KYG40751   
4474  M433DRAFT_532899  KYG40752   
4475  M433DRAFT_159980  KYG40743   

                                               cds_list             5utr_list  \
0                                    [[6014.0, 7120.0]]    [[7121.0, 7339.0]]   
1                                  [[11373.0, 11645.0]]  [[11088.0, 11372.0]]   
2                                  [[13771.0, 14022.0]]  [[13661.0, 13770.0]]   
3              [[14313.0, 14365.0], [14421.0, 14628.0]]  [[13990.0, 14312.0]]   
4                                  [[14742.0, 15704.0]]  [[15705.0, 15905.0]]   
...                                                 ...                   ...

In [8]:


def extract_id(attributes_str):
    attributes = attributes_str.split(';')
    for attr in attributes:
        key_value = attr.split('=')
        if key_value[0] == 'ID':
            return key_value[1]
    return None

def extract_attribute_value(attributes_str, key):
    attributes = attributes_str.split(';')
    for attr in attributes:
        key_value = attr.split('=')
        if key_value[0] == key:
            return key_value[1]
    return None


def extract_longest_names_sacch(gff3):
    '''extract mrna_id to gene_id data + groupby select longest transcript'''
    id_convert = pd.DataFrame(columns=['gene_id','mrna_id','mrna_len','mrna_idx']) #makes new df with the 4 columns
    select = gff3[gff3.type == 'mRNA']  #uses the inputted df and makes a new df (named 'select') where only rows with the type == mRNA are kept, along with all the other columns and data
    id_convert['mrna_id'] = select['attributes'].apply(lambda x: extract_attribute_value(x, 'ID'))
    id_convert['gene_id'] = select['attributes'].apply(lambda x: extract_attribute_value(x, 'Parent'))
    id_convert['mrna_len'] = np.absolute(select.start - select.end) #using the start and end columns in 'select', mrna_len is calculated and deposited into 'id_convert' df
    id_convert['mrna_idx'] = id_convert.index #basically labels the row as to its position in the df, for example first row will have an index of 0, 2nd - 1 and so on
    id_convert = id_convert.reset_index(drop=True) #ensures old index is not added (more of an error preventing line)
    id_convert_idxmax = id_convert.groupby(['gene_id'])['mrna_len'].idxmax() #one gene_id can have multiple mrna rows, this groups them
    id_convert_max = id_convert.iloc[id_convert_idxmax.values] #keeps longest mrna for each gene_id discards the rest 
    
    return id_convert_max

In [9]:
# Parsing GFF3 to regions for Saccharomyces Cerevisiae 
    #'''return regions dframe from gff3 data'''


regions_list = []
        # Filter to keep only the specified feature types
sac_gff3_df = sac_gff3_df[sac_gff3_df.type.isin(['gene', 'mRNA', 'three_prime_UTR', 'exon', 'CDS', 'five_prime_UTR'])]
    
# Initialize
id_convert_max = extract_longest_names_sacch(sac_gff3_df)  # Extracts the longest mRNA entries
dframe = pd.DataFrame(columns=['gene_id', 'mrna_id', 'cds_list', '5utr_list', '3utr_list', 'ostart', 'oend', 'strand', 'seq_id'])
    
# Assign 'id' based on the 'ID' attribute, or use 'Parent' for CDS to align with mRNA IDs
sac_gff3_df['id'] = sac_gff3_df.attributes.apply(lambda x: extract_id(x))
        
# Replace CDS ids with their parent transcript ids for consistency
sac_gff3_df.loc[sac_gff3_df['type'] == 'CDS', 'id'] = sac_gff3_df[sac_gff3_df['type'] == 'CDS'].attributes.apply(lambda x: extract_attribute_value(x, 'Parent'))
        
# Also extract 'Parent' IDs for later filtering (ensure this column is added before running the below line)
sac_gff3_df['parent_id'] = sac_gff3_df.attributes.apply(lambda x: extract_attribute_value(x, 'Parent'))
feature_types_to_update = ['CDS', 'five_prime_UTR', 'three_prime_UTR', 'exon']
for feature_type in feature_types_to_update:
     sac_gff3_df.loc[sac_gff3_df['type'] == feature_type, 'id'] = sac_gff3_df.loc[sac_gff3_df['type'] == feature_type, 'parent_id']
        
# Adjust the filtering of gff3 to create gff3_max, including UTRs based on parent-child relationship
gff3_max = sac_gff3_df[(sac_gff3_df['id'].isin(id_convert_max.mrna_id)) |
            ((sac_gff3_df['parent_id'].isin(id_convert_max.mrna_id)) & sac_gff3_df['type'].isin(['five_prime_UTR', 'three_prime_UTR']))]
        
        

dframe['ostart'] = gff3_max[gff3_max.type == 'mRNA'].start
dframe['oend'] = gff3_max[gff3_max.type == 'mRNA'].end
dframe['strand'] = gff3_max[gff3_max.type == 'mRNA'].strand
dframe['seq_id'] = gff3_max[gff3_max.type == 'mRNA'].seq_id
 # extract ids, convert gene_id from mrna_id
dframe['mrna_id'] = gff3_max[gff3_max.type == 'mRNA'].id
dframe['gene_id'] = id_convert_max.set_index(['mrna_id']).loc[dframe.mrna_id.values].gene_id.values
        # make id from attributes
        # use function to list to obtain lists of CDS and UTRs
        # Obtain the DataFrame with UTR lists and ensure 'mrna_id' is a column
        
           

        
        
dframe['cds_list'] = group_regions_list(gff3_max,'CDS').loc[dframe.mrna_id].values
five_prime_utr_series = group_regions_list(gff3_max, 'five_prime_UTR')# Map the mrna_id in dframe to this Series, filling with NaN where there's no match
dframe['5utr_list'] = dframe['mrna_id'].map(five_prime_utr_series)
three_prime_utr_series = group_regions_list(gff3_max, 'three_prime_UTR')
dframe['3utr_list'] = dframe['mrna_id'].map(three_prime_utr_series)
five_prime_utr_series = group_regions_list(gff3_max, 'five_prime_UTR')
        
        
  
        
         # clear up rows with empty cds - these are not genes
dframe.dropna(subset=['cds_list','ostart','oend','5utr_list','3utr_list'],inplace=True)
        
        # promoter and terminator lists
        # define boundary lists, based on strand
        # divide frame into positive and negative strand = vectorized on both halves
len_prom = int(1000)
len_term = int(500)
len_prom_f = [int(1000),int(500)]
len_term_f = [int(500),int(500)]
# positive strand
ostart = dframe[dframe.strand=='+'].ostart
oend = dframe[dframe.strand=='+'].oend
dframe.loc[dframe.strand=='+','prom_list'] = pd.concat([ostart-len_prom+1,ostart],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
dframe.loc[dframe.strand=='+','term_list'] = pd.concat([oend,oend+len_term-1],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
dframe.loc[dframe.strand=='+','prom_full_list'] = pd.concat([ostart-len_prom_f[0]+1,ostart+len_prom_f[1]],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
dframe.loc[dframe.strand=='+','term_full_list'] = pd.concat([oend-len_term_f[0],oend+len_term_f[1]-1],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
        # negative strand reversed names and sizes
ostart = dframe[dframe.strand=='-'].ostart
oend = dframe[dframe.strand=='-'].oend
dframe.loc[dframe.strand=='-','term_list'] = pd.concat([ostart-len_term+1,ostart],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
dframe.loc[dframe.strand=='-','prom_list'] = pd.concat([oend,oend+len_prom-1],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
dframe.loc[dframe.strand=='-','term_full_list'] = pd.concat([ostart-len_term_f[1]+1,ostart+len_term_f[0]],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])
dframe.loc[dframe.strand=='-','prom_full_list'] = pd.concat([oend-len_prom_f[1],oend+len_prom_f[0]-1],axis=1).apply(lambda x: x.values,axis=1).apply(lambda x: [x])

        # calculate lengths of regions
dframe['cds_len'] = dframe['cds_list'].apply(lambda x: sum(np.diff(x))[0])
dframe['5utr_len'] = dframe['5utr_list'].apply(lambda x: 0 if np.any(pd.isna(x)) else sum(np.diff(x))[0])
dframe['3utr_len'] = dframe['3utr_list'].apply(lambda x: 0 if np.any(pd.isna(x)) else sum(np.diff(x))[0])
        
        # remove regions above limit for speed
dframe = dframe[(dframe.cds_len < 20000) & (dframe['5utr_len'] < 10000) & (dframe['3utr_len'] < 10000)].reset_index(drop=True)
        
        # assertions 
        #print(len(dframe['gene_id'].unique()) == len(dframe['gene_id']))
        #print(dframe.isnull().values.any())
        
regions_list.append(dframe)
    
Sacch_dataframe = regions_list[0]


#print(type(Sacch_dataframe))
#print(Sacch_dataframe)
# Save the DataFrame to a CSV file in the current working directory
#Sacch_dataframe.to_csv("Sacch_df.csv", index=False)


In [10]:
#INSERTING SACCHAROMYCES CEREVISIAE MANUALLY 


species_dataframes.insert(29, Sacch_dataframe)



# Now species_names has 33 elements, and 'New_Species' is at index 30
print(f"After insertion, length of list: {len(species_dataframes)}")
print(species_dataframes[29])
print(len(species_dataframes))
print(len(species_names))

After insertion, length of list: 33
        gene_id          mrna_id                cds_list  \
0       YAL067C    YAL067C_id001      [[7235.0, 9016.0]]   
1       YAL066W    YAL066W_id001    [[10091.0, 10399.0]]   
2     YAL064W-B  YAL064W-B_id001    [[12046.0, 12426.0]]   
3     YAL063C-A  YAL063C-A_id001    [[22395.0, 22685.0]]   
4       YAL062W    YAL062W_id001    [[31567.0, 32940.0]]   
...         ...              ...                     ...   
5189    YPR197C    YPR197C_id001  [[933898.0, 934461.0]]   
5190    YPR198W    YPR198W_id001  [[934034.0, 935665.0]]   
5191    YPR199C    YPR199C_id001  [[938148.0, 939032.0]]   
5192    YPR200C    YPR200C_id001  [[939279.0, 939671.0]]   
5193    YPR201W    YPR201W_id001  [[939922.0, 941136.0]]   

                   5utr_list               3utr_list    ostart      oend  \
0         [[9017.0, 9049.0]]      [[7013.0, 7234.0]]    7013.0    9049.0   
1        [[9807.0, 10090.0]]    [[10400.0, 10460.0]]    9807.0   10460.0   
2       [[11313

In [11]:
def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} # makes a dictionary named 'complement', maps each base to its complement
    bases = list(seq) 
    bases = reversed([complement.get(base,base) for base in bases])
    bases = ''.join(bases)
    return bases




In [12]:
def extract_region_from_list(chrx,region,strand): # region =  list of tuples/lists where each tuple/list represents the start and end positions of an exon
    '''concatenates exons'''                      # strand (a string indicating the DNA strand, either '+' for the positive strand or '-' for the negative strand).
    # fixes discrepancy between pyfaidx/python and gff indexing
    # adapted to case that list values not int type
    seq_list = []
    
    # check if any nan
    if not np.any(pd.isna(region)) and not not region[0][0]:
        for i in range(len(region)):
            if (int(region[i][1])-int(region[i][0])) > 0:
                seq_list.append(str(chrx[int(region[i][0])-1:int(region[i][1])]))
            else:
                seq_list.append('')
        out = ''.join(seq_list)
        if strand in '-':
            out = reverse_complement(out)
    else:
        out = ''
    return str(out)

In [13]:
def GC_content(seq):
    if len(seq) > 0:
        output = sum([i in ['G','C'] for i in seq])/len(seq)
    else:
        output = 0
    return output


In [14]:
from Bio import SeqIO
def extract_all_orf_seqs(species_names, species_dframes, folder):
    '''extract from fasta - load, extract, return dframe'''
    '''script also returns UTR GC contents'''
    all_orf_seqs = []
    all_orf_vars = []
    for i, species_name in enumerate(species_names):
        species_dframe = species_dataframes[i]
        fasta_file = os.path.join(folder, species_name + '.fa')
   
            # initilaize
        len_utr5 = int(300)
        len_utr3 = int(350)
        len_prom = int(1000)
        len_term = int(500)
        len_prom_f = [int(1000),int(500)]
        len_term_f = [int(500),int(500)]

    
        chrx = Fasta(folder+'/'+species_name + '.fa')
        # build fasta files as a dictionary for easier indexing

    
        orf_data = species_dframe.copy(deep=True)
        orf_data.reset_index(drop=True,inplace=True) # just in case
    
        # based on input dframe with boundaries extract genes
        orf_seqs = pd.DataFrame(columns=['gene_id','prom','prom_full','5UTR','coding','3UTR','term','term_full'])
        orf_vars = pd.DataFrame(columns=['gene_id','prom','prom_full','5UTR',
                                  '3UTR','term','term_full','len_cd',
                                  'len_5u','len_3u','freq_cd','GC1',
                                  'GC2','GC3','gc_5u','gc_3u'])
    
        for k, row in orf_data.iterrows():
            #print(k)

            # do not include genes from mitochondrial chromosome 'mt'
            chr_num = str(row.seq_id)
            if not(chr_num == 'mt'): # can be omitted
            
                # edge cases on chromosomes - ignore these few genes
                if orf_data.at[k,'strand'] == '+':
                    ostart = int(orf_data.at[k,'ostart'])
                    oend = int(orf_data.at[k,'oend'])
                    if ((ostart-len_prom)<0 or (oend+len_term)>len(chrx[chr_num])):
                        #print('out of bounds at sequence start')
                        continue
                else:
                    ostart = int(orf_data.at[k,'oend'])
                    oend = int(orf_data.at[k,'ostart'])
                    if ((ostart+len_prom)>len(chrx[chr_num]) or (oend-len_term)<0):
                        #print('out of bounds at sequence end')
                        continue
            
                # save lengths to final data frame
                orf_seqs.at[k,'gene_id'] = row['gene_id']
                orf_vars.at[k,'gene_id'] = row['gene_id']
                orf_vars.at[k,'len_5u'] = row['5utr_len']
                orf_vars.at[k,'len_3u'] = row['3utr_len']
                
                orf_seqs.at[k,'coding'] = extract_region_from_list(chrx[chr_num],row['cds_list'],row['strand'])
                orf_seqs.at[k,'prom'] = extract_region_from_list(chrx[chr_num],row['prom_list'],row['strand'])
                orf_seqs.at[k,'prom_full'] = extract_region_from_list(chrx[chr_num],row['prom_full_list'],row['strand'])
                orf_seqs.at[k,'term'] = extract_region_from_list(chrx[chr_num],row['term_list'],row['strand'])
                orf_seqs.at[k,'term_full'] = extract_region_from_list(chrx[chr_num],row['term_full_list'],row['strand'])

                tmp_5u = extract_region_from_list(chrx[chr_num],row['5utr_list'],row['strand']) 
                tmp_3u = extract_region_from_list(chrx[chr_num],row['3utr_list'],row['strand']) 
                orf_seqs.at[k,'5UTR'] = tmp_5u[-len_utr5:]
                orf_seqs.at[k,'3UTR'] = tmp_3u[-len_utr3:]
                orf_vars.at[k,'gc_5u'] = GC_content(str(tmp_5u)) 
                orf_vars.at[k,'gc_3u'] = GC_content(str(tmp_3u)) 

        
        orf_seqs = orf_seqs.dropna().reset_index(drop=True)
        print(species_name)
        print(orf_seqs.shape)
        orf_vars = orf_vars.dropna(subset=['gene_id']).reset_index(drop=True)               
        print(orf_vars.shape)
        all_orf_seqs.append(orf_seqs)
        all_orf_vars.append(orf_vars)

    return all_orf_seqs, all_orf_vars



all_orf_seqs, all_orf_vars = extract_all_orf_seqs(species_names, species_dataframes, 'fasta')


Acidomyces_richmondensis
(3697, 8)
(3697, 16)
Acidomyces_sp
(3511, 8)
(3511, 16)
Alternaria_alternata
(7467, 8)
(7467, 16)
Ascoidea_rubescens
(2847, 8)
(2847, 16)
Aspergillus_brasiliensis
(5268, 8)
(5268, 16)
Aspergillus_candidus
(6199, 8)
(6199, 16)
Aspergillus_carbonarius
(7038, 8)
(7038, 16)
Aspergillus_glaucus
(5241, 8)
(5241, 16)
Aspergillus_luchuensis
(5706, 8)
(5706, 16)
Aspergillus_novofumigatus
(5281, 8)
(5281, 16)
Aspergillus_ochraceoroseus
(4392, 8)
(4392, 16)
Aspergillus_sydowi
(5060, 8)
(5060, 16)
Aspergillus_tubingensis
(4544, 8)
(4544, 16)
Aspergillus_versicolor
(5342, 8)
(5342, 16)
Aspergillus_wentii
(5170, 8)
(5170, 16)
Babjeviella_inositovora
(3712, 8)
(3712, 16)
Coniochaeta_ligniaria
(6688, 8)
(6688, 16)
Cyberlindnera_jadinii
(3004, 8)
(3004, 16)
Daldinia_sp
(5825, 8)
(5825, 16)
Hyphopichia_burtonii
(2503, 8)
(2503, 16)
Hypoxylon_sp
(5680, 8)
(5680, 16)
Metschnikowia_bicuspidata
(1925, 8)
(1925, 16)
Microdochium_bolley
(7122, 8)
(7122, 16)
Nadsonia_fulvescens
(2340, 

In [15]:
def return_ATG_codons(seqs):
    '''return ATG codons filtered coding sequences'''
    print(pd.Series.value_counts(seqs['coding'].str.slice(0,3)).head())
    return seqs['coding'].str.slice(0,3) == 'ATG'




In [16]:
def count_codons(file):
    '''codon frequency counter'''
    CodonsDict = {
        'TTT': 0, 'TTC': 0, 'TTA': 0, 'TTG': 0, 'CTT': 0,
        'CTC': 0, 'CTA': 0, 'CTG': 0, 'ATT': 0, 'ATC': 0,
        'ATA': 0, 'ATG': 0, 'GTT': 0, 'GTC': 0, 'GTA': 0,
        'GTG': 0, 'TAT': 0, 'TAC': 0, 'TAA': 0, 'TAG': 0,
        'CAT': 0, 'CAC': 0, 'CAA': 0, 'CAG': 0, 'AAT': 0,
        'AAC': 0, 'AAA': 0, 'AAG': 0, 'GAT': 0, 'GAC': 0,
        'GAA': 0, 'GAG': 0, 'TCT': 0, 'TCC': 0, 'TCA': 0,
        'TCG': 0, 'CCT': 0, 'CCC': 0, 'CCA': 0, 'CCG': 0,
        'ACT': 0, 'ACC': 0, 'ACA': 0, 'ACG': 0, 'GCT': 0,
        'GCC': 0, 'GCA': 0, 'GCG': 0, 'TGT': 0, 'TGC': 0,
        'TGA': 0, 'TGG': 0, 'CGT': 0, 'CGC': 0, 'CGA': 0,
        'CGG': 0, 'AGT': 0, 'AGC': 0, 'AGA': 0, 'AGG': 0,
        'GGT': 0, 'GGC': 0, 'GGA': 0, 'GGG': 0}
    
    # make the codon dictionary local
    codon_count = CodonsDict.copy()
    # iterate over sequence and count all the codons in the string.
    # make sure the sequence is upper case
    if str(file).islower():
        dna_sequence = str(file).upper()
    else:
        dna_sequence = str(file)
    for i in range(0, len(dna_sequence), 3):
        codon = dna_sequence[i:i + 3]
        if codon in codon_count:
            codon_count[codon] += 1
        #else:
            #raise TypeError("illegal codon %s" % (codon))
            #print("illegal codon %s" % (codon))
    # return values in dict with sorted keys alphabetically
    out=list()
    for key,value in sorted(codon_count.items()):
        out.append(value)
    
    return np.asarray(out)



In [17]:
def count_codon_GC(file):
    out1=list()
    out2=list()
    out3=list()
    for i in range(0,len(file)-2,3):
        out1.append(int(file[i] in ['G','C']))
        out2.append(int(file[i+1] in ['G','C']))
        out3.append(int(file[i+2] in ['G','C']))
    return 3*sum(out1)/len(file),3*sum(out2)/len(file),3*sum(out3)/len(file)

In [18]:
def NT_to_onehot(data):
    alphabet = 'ACGT'
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    integer_encoded = []
    for char in data:
        if char in alphabet:
            integer_encoded.append(char_to_int[char])
        else:
            integer_encoded.append(0)
    onehot = np.array(to_categorical(integer_encoded,num_classes=4),dtype=np.int8)
    onehot[[char not in alphabet for char in data]] = [0,0,0,0]
    return onehot

def onehot_to_NT(input):
    # input is whole Xhot array
    bases = np.array(['A','C','G','T'])
    output = []
    for i in input:
        tmp = np.concatenate([np.zeros((i.shape[0],1)),i],axis=1).astype('bool')
        output.append(''.join(bases[np.argmax(tmp,axis = 1)]))
    return output

def makeNucSeq(input):
    # input is whole Xhot array
    bases = np.array(['N','A','C','G','T'])
    output = []
    for i in input:
        tmp = np.concatenate([np.zeros((i.shape[0],1)),i],axis=1).astype('bool')
        output.append(''.join(bases[np.argmax(tmp,axis = 1)]))
    return output



In [19]:

def zero_padding(inp,length,start):
    # zero pad input one hot matrix to desired length
    # start .. boolean if pad start of sequence (True) or end (False)
    assert len(inp) < length
    out = np.zeros((length,inp.shape[1]))
    if start:
        out[-inp.shape[0]:] = inp
    else:
        out[0:inp.shape[0]] = inp
    return out



In [20]:
def process_all_onehot_vars(orf_vars,seqs):
    '''processes data to one hot and additional variables'''
    updated_all_orf_vars = []
    for orf_vars, seqs in zip(all_orf_vars, all_orf_seqs):
        len_utr5 = int(300)
        len_utr3 = int(350)

        for k in range(0,seqs.shape[0]):
            orf_vars.at[k,'freq_cd'] = count_codons(seqs.at[k,'coding'])
            orf_vars.at[k,'len_cd'] = len(seqs.at[k,'coding'])
            orf_vars.at[k,'GC1'],orf_vars.at[k,'GC2'],orf_vars.at[k,'GC3'] = count_codon_GC(seqs.at[k,'coding'])

            # one hot
            orf_vars.at[k,'prom'] = NT_to_onehot(seqs.at[k,'prom'])
            orf_vars.at[k,'term'] = NT_to_onehot(seqs.at[k,'term'])
            orf_vars.at[k,'prom_full'] = NT_to_onehot(seqs.at[k,'prom_full'])
            orf_vars.at[k,'term_full'] = NT_to_onehot(seqs.at[k,'term_full'])

            # zero padding
            assert (len(seqs.at[k,'5UTR']) <= len_utr5)
            if len(seqs.at[k,'5UTR']) == len_utr5: # no zero padding 
                orf_vars.at[k,'5UTR'] = NT_to_onehot(seqs.at[k,'5UTR'])
            elif len(seqs.at[k,'5UTR']) > 0: # zero padding
                orf_vars.at[k,'5UTR'] = zero_padding(NT_to_onehot(seqs.at[k,'5UTR']),len_utr5,True)
            else: # pass np.zeros(1,4) to zero padding to 100
                orf_vars.at[k,'5UTR'] = zero_padding(np.zeros((1,4)),len_utr5,True)

            assert (len(seqs.at[k,'3UTR']) <= len_utr3)                                                
            if len(seqs.at[k,'3UTR']) == len_utr3:
                orf_vars.at[k,'3UTR'] = NT_to_onehot(seqs.at[k,'3UTR'])
            elif len(seqs.at[k,'3UTR']) > 0: # zero padding
                orf_vars.at[k,'3UTR'] = zero_padding(NT_to_onehot(seqs.at[k,'3UTR']),len_utr3,True)
            else:
                orf_vars.at[k,'3UTR'] = zero_padding(np.zeros((1,4)),len_utr3,True)

        # assertions 
       # print(len(orf_vars['gene_id'].unique()) == len(orf_vars['gene_id']))
        #print(len(orf_vars['gene_id']))
        #print(orf_vars.isnull().values.any())

        updated_all_orf_vars.append(orf_vars)

    return updated_all_orf_vars


updated_all_orf_vars = process_all_onehot_vars(all_orf_vars, all_orf_seqs)


In [22]:
def generate_all_output_arrays(updated_all_orf_vars):
    '''make separate output arrays for one_hots and variables'''
    all_Xhot = []
    all_Xhot_full = []
    all_Xval = []

    for orf_vars in updated_all_orf_vars:
        Xhot = []
        Xhot_full = []
        # Initialize Xval for each species based on the current orf_vars DataFrame
        Xval = np.zeros((orf_vars.shape[0], 72))

    
    
        for i in range(0,len(orf_vars)):
            tmp = []
            tmp.append(np.asarray(orf_vars.at[i,'prom'],dtype=np.int8))
            tmp.append(np.asarray(orf_vars.at[i,'5UTR'],dtype=np.int8))
            tmp.append(np.asarray(orf_vars.at[i,'3UTR'],dtype=np.int8))
            tmp.append(np.asarray(orf_vars.at[i,'term'],dtype=np.int8))
            Xhot.append(tmp)

            tmp = []
            tmp.append(np.asarray(orf_vars.at[i,'prom_full'],dtype=np.int8))
            tmp.append(np.asarray(orf_vars.at[i,'5UTR'],dtype=np.int8))
            tmp.append(np.asarray(orf_vars.at[i,'3UTR'],dtype=np.int8))
            tmp.append(np.asarray(orf_vars.at[i,'term_full'],dtype=np.int8))
            Xhot_full.append(tmp)

            Xval[i,0] = np.int16(orf_vars.at[i,'len_5u'])
            Xval[i,1] = np.int16(orf_vars.at[i,'len_cd'])
            Xval[i,2] = np.int16(orf_vars.at[i,'len_3u'])
            Xval[i,3] = np.int16(orf_vars.at[i,'gc_5u']*1000)
            Xval[i,4] = np.int16(orf_vars.at[i,'gc_3u']*1000)
            Xval[i,5] = np.int16(orf_vars.at[i,'GC1']*1000)
            Xval[i,6] = np.int16(orf_vars.at[i,'GC2']*1000)
            Xval[i,7] = np.int16(orf_vars.at[i,'GC3']*1000)
            Xval[i,8:] = np.asarray(orf_vars.at[i,'freq_cd'],dtype=np.int16)

        all_Xhot.append(Xhot)
        all_Xhot_full.append(Xhot_full)
        all_Xval.append(Xval)

    
    return all_Xhot, all_Xhot_full, all_Xval

all_Xhot, all_Xhot_full, all_Xval = generate_all_output_arrays(updated_all_orf_vars)




In [43]:
# Formatting and saving Xhot data to be in a suitable format for the model

def concatenate_orf_sequences(orf):
    # orf is a list with four elements: prom, 5UTR, 3UTR, term
    # Each element is a one-hot encoded sequence with shapes like (1000, 4), (300, 4), etc.
    concatenated_sequence = np.concatenate(orf, axis=0)
    return concatenated_sequence


output_dir = 'Species_Xhot'

# Iterate over each species and its name in all_Xhot and species_names respectively
for species_array, species_name in zip(all_Xhot, species_names):
    concatenated_sequences_species = []
    
    # Iterate over each ORF in the current species and concatenate
    for orf in species_array:
        concatenated_sequence = concatenate_orf_sequences(orf)
        concatenated_sequences_species.append(concatenated_sequence)
    
    # Convert the list of concatenated sequences for the current species into a 3D NumPy array
    species_array = np.array(concatenated_sequences_species)
    print(species_array.shape)
    # Save the array to a file within 'species_Xhot1', naming the file after the species
    file_path = os.path.join(output_dir, f"{species_name}_Xhot.npy")
    np.save(file_path, species_array)

# At this point, each species' array has been saved as a .npy file within the folder 'species_Xhot'


(3697, 2150, 4)
(3511, 2150, 4)
(7467, 2150, 4)
(2847, 2150, 4)
(5268, 2150, 4)
(6199, 2150, 4)
(7038, 2150, 4)
(5241, 2150, 4)
(5706, 2150, 4)
(5281, 2150, 4)
(4392, 2150, 4)
(5060, 2150, 4)
(4544, 2150, 4)
(5342, 2150, 4)
(5170, 2150, 4)
(3712, 2150, 4)
(6688, 2150, 4)
(3004, 2150, 4)
(5825, 2150, 4)
(2503, 2150, 4)
(5680, 2150, 4)
(1925, 2150, 4)
(7122, 2150, 4)
(2340, 2150, 4)
(7988, 2150, 4)
(4691, 2150, 4)
(9633, 2150, 4)
(5249, 2150, 4)
(8563, 2150, 4)
(5194, 2150, 4)
(8687, 2150, 4)
(1636, 2150, 4)
(5165, 2150, 4)


In [44]:
import numpy as np
import os

def save_Xval_arrays(all_Xval, species_names):
    # Ensure the output folder exists
    output_folder = 'Species_Xval'
    os.makedirs(output_folder, exist_ok=True)
    
    # Iterate through each Xval array and corresponding species name
    for Xval, species_name in zip(all_Xval, species_names):
        # Convert the list of arrays (Xval) for the species into a NumPy array
        # Note: This requires that all elements in Xval can be converted directly to a NumPy array.
        # If the sequences within Xval have varying lengths, this step may need adjustment,
        # such as padding sequences to a common length.
        Xval_array = np.array(Xval, dtype=object)
        
        # Construct the filename and save path
        save_path = os.path.join(output_folder, f'{species_name}_Xval.npy')
        print(Xval.dtype)
        # Save the array to a file
        np.save(save_path, Xval_array)

# Example call
# Make sure you replace 'all_Xval' and 'species_names' with your actual data
save_Xval_arrays(all_Xval, species_names)


float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64


In [17]:
import pandas as pd

def add_id_columns(gff_df):
    # Helper function to extract IDs from attributes column
    def get_id(attributes, key):
        # Ensure attributes is a string before attempting to split
        if pd.isna(attributes):
            return None
        pairs = str(attributes).split(';')
        id_pair = next((s for s in pairs if s.startswith(f'{key}=')), None)
        if id_pair:
            try:
                return id_pair.split('=')[1].split(':')[1]
            except IndexError:
                # Handle cases where split results don't have a second part
                return None
        else:
            return None

    # Apply the helper function to the 'attributes' column for mRNA ID and Gene ID
    gff_df['mrna_id'] = gff_df['attributes'].apply(lambda attr: get_id(attr, 'ID'))
    gff_df['gene_id'] = gff_df['attributes'].apply(lambda attr: get_id(attr, 'Parent'))
    
    return gff_df

# Assuming gff3_list is your list of GFF DataFrame objects
new_gff3_list = []  # Initialize new_gff3_list as an empty list

# Loop through each DataFrame in gff3_list and apply add_id_columns
for gff_df in gff3_list:
    modified_gff_df = add_id_columns(gff_df)
    new_gff3_list.append(modified_gff_df)

# Now, new_gff3_list should contain the modified DataFrames
print(new_gff3_list)


[             seq_id                             source             type  \
0         KYG50636                                 NaN              NaN   
1        scaffold_1  Acidomyces_richmondensis_BFW_v1.0      supercontig   
2        scaffold_1                                ena             gene   
3        scaffold_1                                ena             mRNA   
4        scaffold_1                                ena  three_prime_UTR   
...             ...                                ...              ...   
85518  scaffold_999                                ena             mRNA   
85519  scaffold_999                                ena   five_prime_UTR   
85520  scaffold_999                                ena             exon   
85521  scaffold_999                                ena              CDS   
85522  scaffold_999                                ena  three_prime_UTR   

        start       end score strand phase  \
0         NaN       NaN   NaN    NaN   NaN   
1     

In [18]:
# Placeholder for intron data across all species
all_introns_sizes = []
all_introns_counts = []

# Process each species' GFF3 dataframe
for gff_df in new_gff3_list:
    intron_sizes_species = []
    intron_count_species = []

    # Group 'exon' entries by mRNA transcript and sort
    for mRNA_id, exons_df in gff_df[gff_df['type'] == 'exon'].groupby('gene_id'):
        sorted_exons = exons_df.sort_values(by='start')

        # Calculate introns
        intron_sizes = []
        for i in range(len(sorted_exons) - 1):
            intron_start = sorted_exons.iloc[i]['end'] + 1
            intron_end = sorted_exons.iloc[i + 1]['start'] - 1
            intron_size = intron_end - intron_start + 1

            if intron_size > 0:
                intron_sizes.append(intron_size)
        
        # Collect intron data for this species
        intron_sizes_species.extend(intron_sizes)
        intron_count_species.append(len(intron_sizes))
    
    # Collect intron data for all species
    all_introns_sizes.append(intron_sizes_species)
    all_introns_counts.append(intron_count_species)

# Now you can plot histograms for intron sizes and counts per species using all_introns_sizes and all_introns_counts


In [46]:
print(modified_sac_gff3_df)

       seq_id source  type     start       end score strand phase  \
7        chrI    SGD  gene     335.0     649.0     .      +     .   
8        chrI    SGD  mRNA     335.0     649.0     .      +     .   
9        chrI   AGAT  exon     335.0     649.0     .      +     .   
10       chrI    SGD   CDS     335.0     649.0     .      +     0   
11       chrI    SGD  gene     538.0     792.0     .      +     .   
...       ...    ...   ...       ...       ...   ...    ...   ...   
63003  chrXVI    SGD   CDS  944603.0  947701.0     .      +     0   
63004  chrXVI    SGD  gene  946856.0  947338.0     .      -     .   
63005  chrXVI    SGD  mRNA  946856.0  947338.0     .      -     .   
63006  chrXVI   AGAT  exon  946856.0  947338.0     .      -     .   
63007  chrXVI    SGD   CDS  946856.0  947338.0     .      -     0   

                                              attributes              id  \
7      ID=YAL069W;Name=YAL069W;Note=Dubious open read...         YAL069W   
8       ID=YAL069W_

In [19]:
import pandas as pd

# Placeholder for intron counts across all species
all_introns_data = []

# Process each species' GFF3 dataframe
for gff_df in gff3_list:
    # Group 'exon' entries by mRNA transcript and sort
    for mRNA_id, exons_df in gff_df[gff_df['type'] == 'exon'].groupby('gene_id'):
        sorted_exons = exons_df.sort_values(by='start')

        # Calculate the count of introns
        intron_count = len(sorted_exons) - 1 if len(sorted_exons) > 1 else 0
        
        # Append a tuple of the gene_id and intron count
        all_introns_data.append((mRNA_id, intron_count))

# Create a single DataFrame for all species
intron_count_df = pd.DataFrame(all_introns_data, columns=['gene_id', 'intron_count'])

# Now, intron_count_df is a DataFrame with the gene_id and intron_count
print(intron_count_df)

                   gene_id  intron_count
0       ENSRNAT00053567920             0
1       ENSRNAT00053567922             0
2       ENSRNAT00053567924             0
3       ENSRNAT00053567926             0
4       ENSRNAT00053567928             0
...                    ...           ...
364616            KZF26980             6
364617            KZF26981             3
364618            KZF26982             7
364619            KZF26983             1
364620            KZF26984             1

[364621 rows x 2 columns]


In [20]:
import matplotlib.pyplot as plt

# Calculate the average intron count per gene for each species
average_introns_per_gene = [sum(species_counts) / len(species_counts) for species_counts in all_introns_counts]

# Plotting the average intron count per gene
plt.figure(figsize=(12, 8))
plt.bar(species_names, average_introns_per_gene, color='royalblue')
plt.xlabel('Species')
plt.ylabel('Average Intron Count per Gene')
plt.title('Average Number of Introns per Gene Across Species')
plt.xticks(rotation=90)
plt.tight_layout()  # Adjust layout to make room for species names
plt.show()


ZeroDivisionError: division by zero

In [43]:
import pandas as pd

# Assuming sac_gff_df is your Saccharomyces cerevisiae GFF DataFrame
modified_sac_gff3_df = add_id_columns(sac_gff3_df)

# Now, modified_sac_gff3_df contains the Saccharomyces cerevisiae DataFrame with the added 'mrna_id' and 'gene_id' columns
print(modified_sac_gff3_df.head())  # Print the first few rows to verify the changes


   seq_id source  type  start    end score strand phase  \
7    chrI    SGD  gene  335.0  649.0     .      +     .   
8    chrI    SGD  mRNA  335.0  649.0     .      +     .   
9    chrI   AGAT  exon  335.0  649.0     .      +     .   
10   chrI    SGD   CDS  335.0  649.0     .      +     0   
11   chrI    SGD  gene  538.0  792.0     .      +     .   

                                           attributes            id  \
7   ID=YAL069W;Name=YAL069W;Note=Dubious open read...       YAL069W   
8    ID=YAL069W_mRNA;Parent=YAL069W;Name=YAL069W_mRNA  YAL069W_mRNA   
9   ID=agat-exon-198;Parent=YAL069W_mRNA;Name=YAL0...  YAL069W_mRNA   
10  ID=agat-cds-1;Parent=YAL069W_mRNA;Name=YAL069W...  YAL069W_mRNA   
11  ID=YAL068W-A;Name=YAL068W-A;Note=Dubious open ...     YAL068W-A   

       parent_id mrna_id gene_id  
7           None    None    None  
8        YAL069W    None    None  
9   YAL069W_mRNA    None    None  
10  YAL069W_mRNA    None    None  
11          None    None    None  


In [49]:
import pandas as pd

def extract_ids_from_attributes(modified_sac_gff3_df):
    # Helper function to extract ID and Parent ID
    def get_id_from_attributes(attributes_str, id_type):
        if pd.isnull(attributes_str):
            return None
        attributes = attributes_str.split(';')
        for attribute in attributes:
            key, _, value = attribute.partition('=')
            if key == id_type:
                return value.split(':')[1] if ':' in value else value
        return None

    # Apply helper function to each row in the DataFrame
    modified_sac_gff3_df['mrna_id'] = modified_sac_gff3_df['attributes'].apply(lambda x: get_id_from_attributes(x, 'ID'))
    modified_sac_gff3_df['gene_id'] = modified_sac_gff3_df['attributes'].apply(lambda x: get_id_from_attributes(x, 'Parent'))

    return modified_sac_gff3_df

# Apply the function to your DataFrame
modified_sac_gff3_df = extract_ids_from_attributes(modified_sac_gff3_df)


In [58]:
import pandas as pd

def extract_ids_from_attributes(sac_gff3_df):
    # Helper function to extract ID and Parent ID
    def get_id_from_attributes(attributes_str, id_type):
        if pd.isnull(attributes_str):
            return None
        attributes = attributes_str.split(';')
        for attribute in attributes:
            key, _, value = attribute.partition('=')
            if key == id_type:
                return value.split(':')[1] if ':' in value else value
        return None

    # Apply helper function to each row in the DataFrame
    sac_gff3_df['mrna_id'] = sac_gff3_df['attributes'].apply(lambda x: get_id_from_attributes(x, 'ID'))
    sac_gff3_df['gene_id'] = sac_gff3_df['attributes'].apply(lambda x: get_id_from_attributes(x, 'Parent'))

    return sac_gff3_df

# Apply the function to your DataFrame
sac_gff3_df = extract_ids_from_attributes(sac_gff3_df)


In [57]:
# Assuming gff_df is your DataFrame containing the GFF3 annotations
# Filter the DataFrame to only include exon entries
exons_df = gff_df[gff_df['type'] == 'exon']

# Group by 'Parent' to aggregate exons by their associated mRNA ID
# Count the number of exons for each mRNA
exon_counts_per_mrna = exons_df.groupby('parent_id').size()

# Filter to get only those mRNAs with more than one exon
multi_exon_mrnas = exon_counts_per_mrna[exon_counts_per_mrna > 1]

# Print the mRNA IDs that have multiple exons and their exon counts
print(multi_exon_mrnas)


KeyError: 'parent_id'

In [70]:
def analyse_fasta_quality(fasta_file):
    total_length = 0
    total_n = 0
    gc_count = 0

    for record in SeqIO.parse(fasta_file, "fasta"):
        seq = record.seq
        total_length += len(seq)
        total_n += seq.count('N') + seq.count('n')
        gc_count += seq.count('G') + seq.count('g') + seq.count('C') + seq.count('c')

    n_percentage = (total_n / total_length) * 100 if total_length else 0
    gc_content = (gc_count / total_length) * 100 if total_length else 0

    return total_length, n_percentage, gc_content


In [40]:
##############################################################################################################################
#investigating aspergillus_carbonarius orf_vars

aspergillus_carbonarius_orf_vars = updated_all_orf_vars[6]


# Print the 1593rd row (Python uses 0-based indexing, so we use 1592 to access the 1593rd row)
print(aspergillus_carbonarius_orf_vars.iloc[1593])


gene_id                                      ASPCADRAFT_204623
prom         [[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,...
prom_full    [[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,...
5UTR         [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...
3UTR         [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,...
term         [[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,...
term_full    [[1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [1,...
len_cd                                                    1902
len_5u                                                      26
len_3u                                                    9054
freq_cd      [10, 13, 6, 9, 7, 7, 9, 6, 2, 5, 3, 4, 7, 26, ...
GC1                                                   0.545741
GC2                                                   0.444795
GC3                                                   0.607256
gc_5u                                                 0.518519
gc_3u                                                 0

In [32]:

import os
import pandas as pd

# Assuming you have the list of species names as species_names
# and the DataFrame of updated ORF variables as updated_all_orf_vars


# Create a folder to store the files if it doesn't exist
folder_name = "species_gff3_dataframes"
os.makedirs(folder_name, exist_ok=True)

# Iterate over the DataFrame and corresponding species names
for species_name, df in zip(species_names, species_dataframes):
    # Create a filename based on the species name
    file_name = f"{species_name}.csv"
    
    # Construct the file path
    file_path = os.path.join(folder_name, file_name)
    
    # Write the DataFrame to a CSV file
    df.to_csv(file_path, index=False)

# Inform Your Majesty that the files have been saved accordingly.
print("All updated ORF variables have been saved to the folder 'species_all_orf_vars'.")


All updated ORF variables have been saved to the folder 'species_all_orf_vars'.


In [25]:
print(type(all_orf_seqs[0]))

<class 'pandas.core.frame.DataFrame'>
